In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Now try to set the API key from the environment variable
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_KEY')


In [5]:
from openai import OpenAI
import json
from dotenv import load_dotenv
import os

load_dotenv()

client = OpenAI(api_key=os.getenv('OPENAI_KEY'))

def generate_evaluation_prompt(attack_path):
    evaluation_prompt = f"""
    You are an expert assistant who evaluates the correctness of an attack path. Provide a detailed evaluation in plain text.

    Output to evaluate:
    {attack_path_test_output}
    
 
    Is crucial important that the attack path should generally provide the following information, regardless of how it is formatted/rephrased/additional details given.

    Expected Steps in Attack Path:

    1. Initial Access through VM1 by way of exposed SSH
       - Description: The attacker can gain initial access by brute forcing the SSH port of VM1, which allows traffic on port 22 from any source in the internet.
       - Weakness: Exposed SSH port in the internet, which could lead to a compromise if weak credentials are leveraged.

    2. Privilege Escalation - The attacker compromises the managed identity assigned to VM1
       - Description: The attacker identifies the managed identity assigned to the VM and can now leverage it for further abuse. The attacker identifies that the managed identity has the "Storage Blob Data Owner" role on a storage account that can be used for data tampering, deletion, and exfiltration.
       - Weakness: Over-privileged permission being assigned to a publicly-facing virtual machine.

    3. Data Exfiltration - The attacker exfiltrates data from the storage account
       - Description: The attacker leverages the role of the managed identity to exfiltrate data from the storage account.
       - Weakness: Over-privileged permission being assigned to a publicly-facing virtual machine.

    4. Lateral Movement - Virtual Network Lateral Movement
       - Description: Although no resources could be identified within the virtual network, the attacker can now reach any endpoint within the virtual network and, potentially, perform bral movement.
       - Weakness: Lack of networking segmentation.

    Instructions:
    ## 1.Read and Understand the Attack path
        - Carefully review the provided Attack path
        - Comprehend the structure and format of the information
        - Analyze the implications of each information step-by-step

    ## 2. Reanalyze the Thinking
        - Review your initial analysis and conclusions
        - Consider alternative interpretations or additional insights you may have missed

    ## 3. Identify Initial Access through VM1 by way of exposed SSH
        - Carefully review the provided Attack path if it contains Initial Access through VM1 by way of exposed SSH.
        - Analyze the implications of each information step-by-step.

    ## 4. Reanalyze the Thinking
        - Review your initial analysis and conclusions.
        - Consider alternative interpretations or additional insights you may have missed.

    ## 5. Privilege Escalation
         - Carefully review the provided Attack path if it contains Privilege Escalation.
         - Analyze the implications of each information step-by-step.
    

    ## 6. Reanalyze the Thinking
        - Review your initial analysis and conclusions.
        - Consider alternative interpretations or additional insights you may have missed.

    ## 7. Data Exfiltration
        - Carefully review the provided Attack path if it contains Data Exfiltration.
        - Analyze the implications of each information step-by-step.

    ## 8. Reanalyze the Thinking
        - Review your initial analysis and conclusions.
        - Consider alternative interpretations or additional insights you may have missed.

    ## 9. Lateral Movement
        - Carefully review the provided Attack path if it contains Lateral Movement.
        - Analyze the implications of each information step-by-step.

    ##10. Reanalyze the Thinking
        - Review your initial analysis and conclusions.
        - Consider alternative interpretations or additional insights you may have missed.

    ## 19. Perform Attack path Review
        - Conduct a detailed review of the attack path.
        - Review if the Attack path provided has the information, regardless of how it is formatted/rephrased/additional details given.
 


    Evaluation Criteria:
    For each step in the attack path, evaluate the following:

    1. Accuracy: Does the attack path correctly identify and describe the step, Initial Access through VM1 by way of exposed SSH, Privilege Escalation, Data Exfiltration, Lateral Movement?
    2. Completeness: Are all key elements of the steps covered, including the description and weakness?
    3. Clarity: Is the explanation clear and easy to understand?
    4. Technical Depth: Does it demonstrate a good understanding of the technical aspects involved?


    Response Format:
    For each step, provide:
    - A rating: 1(achived) or 0(not achieved) or 0
    - A brief reasoning of your rating
    - Specific examples or quotes from the attack path to support your evaluation

    After evaluating each step, provide an overall assessment, including the strengths, weaknesses, and suggestions for improvement.
    """
    return evaluation_prompt

def evaluate_attack_path(attack_path):
    prompt = generate_evaluation_prompt(attack_path)
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an expert assistant who evaluates the correctness of an attack path. Provide a detailed evaluation in plain text."},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
            max_tokens=1000
        )

        response_content = response.choices[0].message.content
        return response_content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
attack_path_test_output = """

1. **System-Assigned Managed Identity**:
1. **Compromise of the Virtual Machine (VM):**
   - **SSH Brute Force Attack:** Since the Network Security Group (NSG) allows inbound SSH traffic from any source, an attacker could attempt to brute force the SSH credentials of the VM.
   - **Exploitation of Vulnerabilities:** If the VM's operating system or any installed software has vulnerabilities, an attacker could exploit these to gain access.

2. **Abuse of Managed Identity:**
   - **Privilege Escalation:** If the VM is compromised, the attacker could leverage the system-assigned managed identity to access other Azure resources with the same identity's permissions.
   - **Lateral Movement:** The attacker could use the managed identity to move laterally within the Azure environment, accessing other resources and potentially escalating privileges.

3. **Misuse of High-Privilege Roles:**
   - **Owner Role Abuse:** The user principal `gabriel.necula@frontguardsecurity.com` has the 'Owner' role at the subscription level, which provides full access to all resources. If this account is compromised, the attacker could take complete control of the subscription.
   - **Contributor Role Abuse:** The user principal `testsentinel@frontguardsecurity.com` has the 'Contributor' role at the subscription level, allowing the creation and management of resources. This role could be abused to deploy malicious resources or modify existing ones.


### Mitigation Recommendations:
- **Restrict SSH Access:** Modify the NSG rules to allow SSH access only from specific IP addresses or ranges.
- **Implement Multi-Factor Authentication (MFA):** Ensure that all user and service principal accounts use MFA to reduce the risk of credential compromise.
- **Regularly Rotate Credentials:** Rotate SSH keys and service principal secrets regularly.
- **Monitor and Audit Logs:** Continuously monitor and audit logs for any suspicious activities.
- **Least Privilege Principle:** Assign the minimum necessary permissions to users and service principals.
- **Network Segmentation:** Implement network segmentation to limit the impact of a compromised resource.

By addressing these potential attack vectors and implementing the recommended mitigations, you can significantly enhance the security posture of your Azure environment.
"""

result = evaluate_attack_path(attack_path_test_output)
if result:
    print(result)


### Evaluation of the Provided Attack Path

#### Step 1: Initial Access through VM1 by way of exposed SSH
- **Rating: 1 (achieved)**
- **Reasoning**: The attack path correctly identifies the initial access vector through an exposed SSH port on the VM. It mentions both brute force attacks and exploitation of vulnerabilities as potential methods for gaining access.
- **Specific Examples**:
  - "SSH Brute Force Attack: Since the Network Security Group (NSG) allows inbound SSH traffic from any source, an attacker could attempt to brute force the SSH credentials of the VM."
  - "Exploitation of Vulnerabilities: If the VM's operating system or any installed software has vulnerabilities, an attacker could exploit these to gain access."

#### Step 2: Privilege Escalation
- **Rating: 1 (achieved)**
- **Reasoning**: The attack path correctly identifies the potential for privilege escalation through the abuse of the managed identity assigned to the VM. It explains how the attacker could leverage 

In [3]:


def generate_evaluation_prompt(attack_path):
    evaluation_prompt = f"""
    You are an expert assistant who evaluates the correctness of an attack path. Provide a detailed evaluation in plain text.

    Output to evaluate:
    {attack_path_test_output}
    
 
It is crucial that the attack path should generally provide the following information, regardless of how it is formatted/rephrased/additional details given:

Expected Steps in Attack Path:

1. Initial Access through VM1 by way of exposed SSH
   - Description: The attacker can gain initial access by brute forcing the SSH port of VM1, which allows traffic on port 22 from any source in the internet.
   - Weakness: Exposed SSH port in the internet, which could lead to a compromise if weak credentials are leveraged.

2. Privilege Escalation - The attacker compromises the managed identity assigned to VM1
   - Description: The attacker identifies the managed identity assigned to the VM and can now leverage it for further abuse. The attacker identifies that the managed identity has the "Storage Blob Data Owner" role on a storage account that can be used for data tampering, deletion, and exfiltration.
   - Weakness: Over-privileged permission being assigned to a publicly-facing virtual machine.

3. Data Exfiltration - The attacker exfiltrates data from the storage account
   - Description: The attacker leverages the role of the managed identity to exfiltrate data from the storage account.
   - Weakness: Over-privileged permission being assigned to a publicly-facing virtual machine.

4. Lateral Movement - Virtual Network Lateral Movement
   - Description: Although no resources could be identified within the virtual network, the attacker can now reach any endpoint within the virtual network and, potentially, perform lateral movement.
   - Weakness: Lack of networking segmentation.

Instructions:
1. Read and understand the provided attack path.
2. For each step (Initial Access, Privilege Escalation, Data Exfiltration, Lateral Movement):
   - Carefully review if the attack path contains this step.
   - Analyze the implications of each piece of information.
   - Evaluate the step based on the criteria below.
   - After evaluating, reanalyze your thinking:
     - Review your initial analysis and conclusions.
     - Consider alternative interpretations or additional insights you may have missed.
3. Perform a final overall review of the attack path.

Evaluation Criteria:
For each step in the attack path, evaluate the following:
1. Accuracy: Does the attack path correctly identify and describe the step?
2. Completeness: Are all key elements of the step covered, including the description and weakness?
3. Clarity: Is the explanation clear and easy to understand?
4. Technical Depth: Does it demonstrate a good understanding of the technical aspects involved?

Response Format:
For each step, provide:
- A rating: 1 (achieved) or 0 (not achieved)
- A brief reasoning for your rating
- Specific examples or quotes from the attack path to support your evaluation

After evaluating each step, provide an overall assessment, including the strengths, weaknesses, and suggestions for improvement.
    """
    return evaluation_prompt

def evaluate_attack_path(attack_path):
    prompt = generate_evaluation_prompt(attack_path)
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an expert assistant who evaluates the correctness of an attack path. Provide a detailed evaluation in plain text."},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
            max_tokens=1000
        )

        response_content = response.choices[0].message.content
        return response_content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
attack_path_test_output = """

**High Privilege Role Assignments**:
    - The user principal `gabriel.necula@frontguardsecurity.com` has the `Owner` role at the subscription level, providing high privileges. If the VM is compromised, the attacker could potentially escalate privileges by targeting this user principal.
"""

result = evaluate_attack_path(attack_path_test_output)
if result:
    print(result)






### Evaluation of the Provided Attack Path

#### Step 1: Initial Access through VM1 by way of exposed SSH

- **Rating**: 0 (not achieved)
- **Reasoning**: The provided output does not mention any initial access through VM1 or the exploitation of an exposed SSH port.
- **Supporting Example**: The output only discusses the high privilege role assignment of a user principal and does not cover the initial access step.

#### Step 2: Privilege Escalation - The attacker compromises the managed identity assigned to VM1

- **Rating**: 0 (not achieved)
- **Reasoning**: The provided output does not mention the compromise of a managed identity assigned to VM1 or any privilege escalation steps.
- **Supporting Example**: The output focuses solely on the high privilege role of a user principal and does not address the managed identity or its permissions.

#### Step 3: Data Exfiltration - The attacker exfiltrates data from the storage account

- **Rating**: 0 (not achieved)
- **Reasoning**: The provid